# Running Macro

In this tutorial, we start from a single zone electricity system with four resource clusters: utility scale solar PV, land-based wind power generation, natural gas combined cycle power plants, and electricity storage. 

We consider three commodities: electricity, natural gas, and $\text{CO}_2$. 

Initially, hydrogen is modeled exogenously, adding a constant electricity demand for hydrogen production to the electricity demand time series. In other words, we assume the existence of an electrolyzer that continuously consumes electricity to meet the hydrogen demand.

We model a greenfield scenario with a carbon price of 200$ \$/ton$, i.e., we allow $\text{CO}_2$ emissions with a penalty cost.

***Note: We use the default units in MACRO: MWh for energy vectors, metric tons for other commodities (e.g., $\text{CO}_2$) and dollars for costs***

In [ ]:
using Pkg; Pkg.add("VegaLite")

In [ ]:
using MacroEnergy
using HiGHS
using CSV
using DataFrames
using JSON3
using VegaLite

We first load the inputs:

In [ ]:
case_dir = "one_zone_electricity_only"

In [ ]:
case = MacroEnergy.load_case(case_dir);

Next, we set the optimizer. Note that we are using the open-source LP solver [HiGHS](https://highs.dev/), alternatives include the commercial solvers [Gurobi](https://www.gurobi.com/), [CPLEX](https://www.ibm.com/products/ilog-cplex-optimization-studio), [COPT](https://www.copt.de/).

In [ ]:
optimizer =  HiGHS.Optimizer
optimizer_env = missing
optimizer_attributes = (
    "solver" => "ipm",
    "run_crossover" => "on",
    "ipm_optimality_tolerance" => 1e-4
)
optimizer = MacroEnergy.create_optimizer(optimizer, optimizer_env, optimizer_attributes)

We are now ready to generate the Macro capacity expansion model and solve it. Because Macro is designed to be solved by [high performance decomposition algorithms](https://arxiv.org/abs/2403.02559), the model formulation has a specific block structure that can be exploited by these schemes. In the case of 3 operational sub-periods, the block structure looks like this:

```

![model_structure](images/model_structure.png)

When generating the model, Macro by default decomposes it as described above. The user then has the option to solve it either as a monolithic model or using Benders decomposition. In this first run, we’ll solve the model as a monolithic problem (see `settings/case_settings.json`).

Macro provides a function to generate the Macro model and solve it according to the solution algorithm (in this case, Monolithic) and the optimizer (HiGHS).

In [ ]:
(case, solution) = MacroEnergy.solve_case(case, optimizer);

To output the results in csv files, we can use the following functions:

In [ ]:
results_monolithic_path = joinpath(case_dir, "results_monolithic")
MacroEnergy.write_outputs(results_monolithic_path, case, solution)

The `case` variable contains both the data and the settings for the model run:

```julia
case.systems    # vector of data for each investment period
```
and 
```julia
case.settings
```

Specifically, the case data is organized as a vector of investment periods, with each element containing the data for a given period. In this example, there is only one investment period, so to access the results, we need to retrieve the first element of this vector. 

In [ ]:
period_index = 1

In [ ]:
system = case.systems[period_index];

Once we have a `System` object, to simply visualize the results, we can use the following function:

In [ ]:
# Final capacity
capacity_results = get_optimal_capacity(system)
capacity_results[:, [:commodity, :resource_id, :type, :value]]

In [ ]:
# New capacity
new_capacity_results = get_optimal_new_capacity(system)
new_capacity_results[:, [:commodity, :resource_id, :type, :value]]

In [ ]:
# Retired capacity
retired_capacity_results = get_optimal_retired_capacity(system)
retired_capacity_results[:, [:commodity, :resource_id, :type, :value]]

In [ ]:
# Discounted costs
cost_results = get_optimal_discounted_costs(solution, 1)
select!(cost_results, Not(["commodity", "commodity_subtype", "zone", "resource_id", "component_id", "type"]))

In [ ]:
# Flows
flow_results = get_optimal_flow(system)
MacroEnergy.reshape_wide(flow_results, :time, :component_id, :value)

and the total emissions (in metric tonnes) are:

In [ ]:
co2_node = MacroEnergy.find_node(system.locations, :co2_sink)
MacroEnergy.value(sum(co2_node.operation_expr[:emissions]))

We can also plot the electricity generation results using `VegaLite.jl`:

In [ ]:
plot_time_interval = 3600:3624
natgas_power = MacroEnergy.value.(MacroEnergy.flow(system.assets[2].elec_edge)).data[plot_time_interval] / 1e3;
solar_power = MacroEnergy.value.(MacroEnergy.flow(system.assets[3].edge)).data[plot_time_interval] / 1e3;
wind_power = MacroEnergy.value.(MacroEnergy.flow(system.assets[4].edge)).data[plot_time_interval] / 1e3;

elec_gen = DataFrame(hours=plot_time_interval,
    solar_photovoltaic=solar_power,
    wind_turbine=wind_power,
    natural_gas_fired_combined_cycle=natgas_power,
)

stack_elec_gen = stack(elec_gen, [:natural_gas_fired_combined_cycle, :wind_turbine, :solar_photovoltaic], variable_name=:resource, value_name=:generation);

elc_plot = stack_elec_gen |>
           @vlplot(
    :area,
    x = {:hours, title = "Hours"},
    y = {:generation, title = "Electricity generation (GWh)", stack = :zero},
    color = {"resource:n", scale = {scheme = :category10}},
    width = 400,
    height = 300
)

**Task:** Set a strict net-zero $\text{CO}_2$ cap by removing the slack allowing constraint violation for a penalty. This can be done by deleting the field `price_unmet_policy` from the $\text{CO}_2$ node in file `one_zone_electricity_only/system/nodes.json`

Then, re-run the model with these new inputs and show the capacity results, total system cost, emissions, and plot the generation profiles.

<details>
<summary>Solution</summary>

Open file `one_zone_electricity_only/system/nodes.json`, go to the bottom of the file where the $\text{CO}_2$ node is defined. Remove the lines related to the field `price_unmet_policy`, so that the node definition looks like this:

```json
 {
    "type": "CO2",
    "global_data": {
        "time_interval": "CO2"
    },
    "instance_data": [
        {
            "id": "co2_sink",
            "constraints": {
                "CO2CapConstraint": true
            },
            "rhs_policy": {
                "CO2CapConstraint": 0
            }   
        }
    ]
}
```
Then, you need to re-load the inputs:
```julia
    system = MacroEnergy.load_system("one_zone_electricity_only");
```
generate the MACRO model:
```julia
    model = MacroEnergy.generate_model(system);
```
and solve it:
```julia
    MacroEnergy.set_optimizer(model, HiGHS.Optimizer);
    MacroEnergy.optimize!(model)
```
We can check the results by printing the total system cost:
```julia
    MacroEnergy.objective_value(model)
```
and the new emissions (which should be zero):
```julia
    co2_node_idx = findfirst(isa.(system.locations,Node{CO2}).==1)
    MacroEnergy.value(sum(system.locations[co2_node_idx].operation_expr[:emissions]))
```
Finally, we plot the generation results:
```julia
    plot_time_interval = 3600:3624
    natgas_power =  MacroEnergy.value.(MacroEnergy.flow(system.assets[2].elec_edge)).data[plot_time_interval]/1e3;
    solar_power = MacroEnergy.value.(MacroEnergy.flow(system.assets[3].edge)).data[plot_time_interval]/1e3;
    wind_power = MacroEnergy.value.(MacroEnergy.flow(system.assets[4].edge)).data[plot_time_interval]/1e3;

    elec_gen =  DataFrame( hours = plot_time_interval, 
                    solar_photovoltaic = solar_power,
                    wind_turbine = wind_power,
                    natural_gas_fired_combined_cycle = natgas_power,
                    )

    stack_elec_gen = stack(elec_gen, [:natural_gas_fired_combined_cycle,:wind_turbine,:solar_photovoltaic], variable_name=:resource, value_name=:generation);

    elc_plot = stack_elec_gen |> 
    @vlplot(
        :area,
        x={:hours, title="Hours"},
        y={:generation, title="Electricity generation (GWh)",stack=:zero},
        color={"resource:n", scale={scheme=:category10}},
        width=400,
        height=300
    )
```
</details>



#### Benders Decomposition

To solve the model using Benders decomposition, the user needs to follow a few simple steps:

1. Set the "SolutionAlgorithm" field in the `settings/case_settings.json` file to "Benders".
2. Create or update the `settings/benders_settings.json` file to configure the Benders decomposition algorithm.
3. Since the algorithm can be distributed, define two optimizers: one for solving the master problem and one for the subproblem.
4. All the other steps are the same as in the previous example.

Note that the considered test system includes slacks for the CO2 cap policy and non-served energy demand. This ensures that all subproblems considered by the decomposition algorithm are feasible. If this was not the case, the algorithm would have to generate feasiblity cuts, which can be looser than Benders optimality cuts and might result in slower convergence.

Step 1. and 2. need to be done manually.

Let's re-load the inputs with the new settings:

In [ ]:
case = MacroEnergy.load_case(case_dir);

Then, let's create the two optimizers as mentioned above:

In [ ]:
planning_optimizer=HiGHS.Optimizer
subproblem_optimizer=HiGHS.Optimizer
planning_optimizer_attributes=("solver" => "ipm", "run_crossover" => "on", "ipm_optimality_tolerance" => 1e-6)
subproblem_optimizer_attributes=("solver" => "ipm", "run_crossover" => "on", "ipm_optimality_tolerance" => 1e-6)
optimizer = MacroEnergy.create_optimizer_benders(planning_optimizer, subproblem_optimizer, planning_optimizer_attributes, subproblem_optimizer_attributes)

Finally, let's solve the model using the Benders decomposition algorithm. Macro uses a Julia implementation of the algorithm, available at [MacroEnergySolvers.jl](https://github.com/macroenergy/MacroEnergySolvers.jl).

In [ ]:
(case, solution) = MacroEnergy.solve_case(case, optimizer);

The runtime of Benders decomposition is longer than monolithic for this test system. This is because we are running the algorithm in series without taking advantage of distributed computing. In more realistic settings (for example on a computer cluster), Benders decomposition is expected to be significantly faster than monolithic model solution, especially for larger systems.

The results are written to the `results_benders` directory.

In [ ]:
results_benders_path = joinpath(case_dir, "results_benders")
MacroEnergy.write_outputs(results_benders_path, case, solution)